In [63]:
# importamos librerías
import pandas as pd
from datetime import datetime

# **Carga del archivo**

In [64]:
# cargamos el archivo con los datos
df_raw = pd.read_parquet("../data/data_raw/reservas_hoteles.parquet")

In [65]:
# info del dataframe con los datos
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [4]:
# Hacemos copia del dataframe para trabajar
df = df_raw.copy()

# Cambio columnas con fechas a datetime

In [5]:
# Las columnas de fecha_reserva, inicio_estancia y final_estancia deberían estar en formato fecha. Hacemos una función para transformarlas.
def trans_fecha(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

In [6]:
df = trans_fecha(["fecha_reserva", "inicio_estancia", "final_estancia"], df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

# Fechas estancia

In [7]:
# miramos valores únicos de fechas de inicio y final de estancia porque las necesitaremos para buscar eventos en la API
ini_estancia =df["inicio_estancia"].unique()
ini_estancia

<DatetimeArray>
['2025-03-01 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [8]:
fin_estancia = df["final_estancia"].unique()
fin_estancia

<DatetimeArray>
['2025-03-02 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

Como las fechas son las mismas, podríamos rellenar toda la columna con el mismo valor.

In [9]:
# Rellenamos las columnas del df con las fechas.
df["inicio_estancia"] = ini_estancia[0]
df["final_estancia"] = fin_estancia[0]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15098 non-null  datetime64[ns]
 8   final_estancia   15098 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [11]:
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
3563,a94a0e23-419a-4ded-a29d-137c10a14af0,c1dcb4a9-f2a1-4752-9870-dec39dfcfe99,Juliana,Clemente,juliana.clemente@example.com,False,2025-02-05,2025-03-01,2025-03-02,48,290.03,Gran Hotel Madrid,2.0,Madrid
7846,3fd51add-dc66-458e-8551-e60a8828113b,bcd73e65-544f-4215-846f-68e7507a77d6,Reyes,Marín,reyes.marín@example.com,False,2025-02-05,2025-03-01,2025-03-02,2,405.75,Hotel Rincón Sereno,2.0,Madrid
8508,3c163ab4-5195-4680-9e34-34d6e953b519,ec8cb74e-0f9d-42b7-8f33-585e157e4b79,Reyes,Cabrera,reyes.cabrera@example.com,True,NaT,2025-03-01,2025-03-02,135,NaN,,NaN,
4989,9d2be2b3-b141-47c9-a703-d448418b9f66,1fe0a285-451d-472e-a5c4-306f836c1ceb,Ariel,Vara,ariel.vara@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
11724,f758376e-dd00-41b1-aea3-b3e1695fd6c3,fbd4ab5f-d39a-4bcf-8cb9-57491f5b3228,Lina,Bou,lina.bou@example.com,False,2025-02-09,2025-03-01,2025-03-02,2,308.27,Hotel Puerta del Cielo,2.0,Madrid


# Ciudades

In [12]:
ciudades = df["ciudad"].unique()
ciudades

array(['', 'Madrid'], dtype=object)

In [13]:
# Rellenamos toda la columna con Madrid.
df["ciudad"] = "Madrid"
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
11776,e93d3283-e576-469d-9807-444337d5e9d1,ee082330-85d1-4759-bb41-e1a0eba9e5e5,Gonzalo,Sureda,gonzalo.sureda@example.com,False,2025-02-01,2025-03-01,2025-03-02,6,423.51,Hotel Sol y Luna,1.0,Madrid
5797,1ea8e240-a80d-405a-a477-38236a14db25,8776d5f5-fa92-47f3-93fd-daeb9c11f620,Tomás,Alegre,tomás.alegre@example.com,False,2025-02-09,2025-03-01,2025-03-02,2,53.71,Hotel Vista Alegre,4.0,Madrid
1291,c5356c21-1efe-406f-b960-017f19bba124,5edff970-f98e-49a0-8a27-be85703f878a,Flor,Gomis,flor.gomis@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,252.82,Hotel Torre Dorada,2.0,Madrid
13264,a8a6c06f-76b8-498b-9a58-96caf4521192,c2f67fc6-6953-4822-b20f-f348170ee6a4,Martirio,Valera,martirio.valera@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid
4644,f5ed45b6-71ae-418c-ba6c-ce47b9a59e9e,5d00a540-3cd4-4cf5-8891-eac6ba0083fc,Matilde,Riba,matilde.riba@example.com,False,2025-02-10,2025-03-01,2025-03-02,33,233.78,Hotel Monte Verde,2.0,Madrid


# Duplicados

In [14]:
duplicados = df.duplicated()
duplicados.sum()

np.int64(98)

In [15]:
# Hay 98 filas totalmente duplicadas, las eliminamos
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

# **Transformación**

De cara a cargar los datos en la base de datos vamos a generar unas tablas a partir del df.

In [16]:
# Podemos tener id_hotel duplicados porque más de un cliente puede reservar en el mismo hotel y podemos tener id_cliente duplicados porque el mismo cliente puede haber hecho varias reservas
# lo que no podemos tener repetido es el id_reserva. Lo comprobamos
reservas = df["id_reserva"].duplicated().sum()
reservas

np.int64(0)

## Ciudades

In [17]:
ciudades_limpio = df["ciudad"].unique()
df_ciudades = pd.DataFrame(ciudades_limpio, columns=["nombre_ciudad"])
df_ciudades

,nombre_ciudad
0,Madrid


## Hoteles

In [18]:
# Cantidad de id_hotel
df["id_hotel"].nunique()

29

In [19]:
# Cantidad nombres hotel
df["nombre_hotel"].nunique()

20

Hay discrepancia en la cantidad de valores únicos. Vamos a generar dos tablas de hoteles, una con los hoteles propios y otra con los de la competencia.

### Hoteles propios

In [20]:
info_propios = df[["id_hotel", "nombre_hotel" , "estrellas", "ciudad", "competencia"]][df["competencia"]==False]
info_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
9290,35,Hotel Rincón Sereno,5.0,Madrid,False
3441,49,Palacio del Sol,5.0,Madrid,False
13979,27,Hotel Torre Dorada,4.0,Madrid,False
12503,3,Gran Hotel Madrid,4.0,Madrid,False
2562,14,Hotel Los Almendros,1.0,Madrid,False


In [21]:
info_propios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9828 entries, 2 to 14999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      9828 non-null   int64  
 1   nombre_hotel  9828 non-null   object 
 2   estrellas     9828 non-null   float64
 3   ciudad        9828 non-null   object 
 4   competencia   9828 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 393.5+ KB


In [22]:
# Comprobamos si hay hoteles con más de un id asignado
conteo_hoteles = info_propios.groupby("nombre_hotel")["id_hotel"].nunique().reset_index()
conteo_hoteles

,nombre_hotel,id_hotel
0,Gran Hotel Madrid,19
1,Hotel Brisas del Mar,19
2,Hotel Camino del Sol,19
3,Hotel Costa Azul,19
4,Hotel Encanto Real,19
5,Hotel Jardines del Rey,19
6,Hotel Las Estrellas,19
7,Hotel Los Almendros,19
8,Hotel Luz de Madrid,19
9,Hotel Maravilla Real,19


Todos los hoteles tienen 19 id asignados, lo que no puede ser correcto. Vamos a asociar un único id_hotel por cada nombre_hotel

In [23]:
nombrespropios = list(info_propios["nombre_hotel"].unique())

In [24]:
# Crear una lista de números seriales según la cantidad de hoteles propios
idspropios = list(range(1, len(nombrespropios)+1))
# Asignar números seriales a cada elemento. Lo hacemos como un diccionario para poder hacer un mapeo en el df_propios
hoteles_propios = dict(zip(nombrespropios, idspropios))
hoteles_propios

{'Hotel Monte Verde': 1,
 'Hotel Brisas del Mar': 2,
 'Hotel Camino del Sol': 3,
 'Hotel Puerta del Cielo': 4,
 'Hotel Encanto Real': 5,
 'Palacio del Sol': 6,
 'Hotel Jardines del Rey': 7,
 'Hotel Las Estrellas': 8,
 'Gran Hotel Madrid': 9,
 'Hotel Torre Dorada': 10,
 'Hotel Palacio Imperial': 11,
 'Hotel Luz de Madrid': 12,
 'Hotel Los Almendros': 13,
 'Hotel Sol y Luna': 14,
 'Hotel Mirador Real': 15,
 'Hotel Rincón Sereno': 16,
 'Hotel Vista Alegre': 17,
 'Hotel Costa Azul': 18,
 'Hotel Maravilla Real': 19}

In [25]:
info_propios["id_hotel"] = info_propios["nombre_hotel"].map(hoteles_propios)
info_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
8804,8,Hotel Las Estrellas,2.0,Madrid,False
8521,13,Hotel Los Almendros,3.0,Madrid,False
13218,10,Hotel Torre Dorada,5.0,Madrid,False
9445,3,Hotel Camino del Sol,2.0,Madrid,False
10343,10,Hotel Torre Dorada,1.0,Madrid,False


La columna estrellas se supone que se refiere a la clasificación del hotel, por lo que cada hotel debería tener una clasificación única. Lo comprobamos

In [26]:
conteo_estrellas = info_propios.groupby("nombre_hotel")["estrellas"].nunique().reset_index()
conteo_estrellas

,nombre_hotel,estrellas
0,Gran Hotel Madrid,5
1,Hotel Brisas del Mar,5
2,Hotel Camino del Sol,5
3,Hotel Costa Azul,5
4,Hotel Encanto Real,5
5,Hotel Jardines del Rey,5
6,Hotel Las Estrellas,5
7,Hotel Los Almendros,5
8,Hotel Luz de Madrid,5
9,Hotel Maravilla Real,5


Vemos cada hotel tiene 5 clasificaciones distintas, lo que no tendría sentido. Vamos a suponer que se trata de la valoración de los clientes, por lo que tomaremos el valor medio de cada hotel.

In [27]:
# Cambiamos el nombre de la columna
info_propios.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [28]:
# Calculamos la valoracion media por cada nombre de hotel y redondeamos a 1 decimal
df_valoracion_propios = round(info_propios.groupby("nombre_hotel")["valoracion"].mean().reset_index(),1)
df_valoracion_propios

,nombre_hotel,valoracion
0,Gran Hotel Madrid,3.1
1,Hotel Brisas del Mar,3.1
2,Hotel Camino del Sol,3.1
3,Hotel Costa Azul,3.1
4,Hotel Encanto Real,3.0
5,Hotel Jardines del Rey,2.9
6,Hotel Las Estrellas,2.9
7,Hotel Los Almendros,3.0
8,Hotel Luz de Madrid,3.1
9,Hotel Maravilla Real,3.0


In [29]:
def generar_mapa(col1, col2, dataframe):
    """
    Genera un diccionario a partir de los valores de dos columnas de un DataFrame, generando pares clave:valor por cada fila.

    Parámetros:
    col1 (str): Nombre de la primera columna que se usará como claves del diccionario.
    col2 (str): Nombre de la segunda columna que se usará como valores del diccionario.
    dataframe (pandas.DataFrame): El DataFrame que contiene las columnas a mapear.

    Retorna:
    dict: Un diccionario donde las claves son los valores de la columna col1 y los valores son los valores de la columna col2.
    """
    lista_col1 = []
    lista_col2 = []
    for i in dataframe[col1]:
        lista_col1.append(i)
    for f in dataframe[col2]:
        lista_col2.append(f)

    mapa = dict(zip(lista_col1, lista_col2))
    return mapa

In [30]:
# generamos un diccionario con los nombres de los hoteles y su valoración media
valoracion_propios = generar_mapa("nombre_hotel", "valoracion", df_valoracion_propios)

In [31]:
# Usamos el diccionario para mapear el df de hoteles propios y corregir la columna de estrellas
info_propios["valoracion"] = info_propios["nombre_hotel"].map(valoracion_propios)
info_propios

,id_hotel,nombre_hotel,valoracion,ciudad,competencia
2,1,Hotel Monte Verde,3.1,Madrid,False
3,2,Hotel Brisas del Mar,3.1,Madrid,False
4,3,Hotel Camino del Sol,3.1,Madrid,False
5,4,Hotel Puerta del Cielo,3.0,Madrid,False
6,5,Hotel Encanto Real,3.0,Madrid,False
...,...,...,...,...,...
14992,7,Hotel Jardines del Rey,2.9,Madrid,False
14994,16,Hotel Rincón Sereno,3.0,Madrid,False
14995,7,Hotel Jardines del Rey,2.9,Madrid,False
14997,2,Hotel Brisas del Mar,3.1,Madrid,False


In [32]:
# usamos los diccionarios que hemos creado para generar un df con los hoteles propios
df_propios = pd.DataFrame(hoteles_propios.items(), columns=["nombre_hotel", "id_hotel"])
# Creamos las columnas que nos faltan para rellenarlas
df_propios[["valoracion", "ciudad", "competencia"]] = pd.NA
df_propios["valoracion"] = df_propios["nombre_hotel"].map(valoracion_propios)
valor_ciudad = "Madrid"
valor_competencia = False
df_propios["ciudad"] = valor_ciudad
df_propios["competencia"] = valor_competencia

### Hoteles competencia

In [33]:
competencia = df[["id_hotel", "nombre_hotel", "estrellas", "ciudad", "competencia"]][df["competencia"]==True]
competencia.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
2705,113,,NaN,Madrid,True
13266,194,,NaN,Madrid,True
1977,114,,NaN,Madrid,True
9890,186,,NaN,Madrid,True
13363,186,,NaN,Madrid,True


In [34]:
# Cambiamos el nombre de la columna
competencia.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [35]:
competencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5172 entries, 0 to 14998
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      5172 non-null   int64  
 1   nombre_hotel  5172 non-null   object 
 2   valoracion    0 non-null      float64
 3   ciudad        5172 non-null   object 
 4   competencia   5172 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 207.1+ KB


In [36]:
ids_competencia = (competencia["id_hotel"].unique()).tolist()

Los datos que faltan los extraemos por scrapeo

In [37]:
info_competencia = pd.read_pickle("hoteles_competencia.pkl")
info_competencia

,nombre,valoracion,precio,fecha_reserva
0,ibis Styles Madrid Prado,4.7,179.0,2025-02-20
1,ibis budget Madrid Calle 30,4.4,106.0,2025-02-20
2,ibis Madrid Centro las Ventas,4.5,181.0,2025-02-20
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-20
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-20
5,ibis Madrid Aeropuerto Barajas,4.4,129.0,2025-02-20
6,ibis Madrid Alcorcon Tresaguas,4.4,95.0,2025-02-20
7,ibis budget Madrid Aeropuerto,4.0,97.0,2025-02-20
8,ibis Madrid Alcobendas,4.4,89.0,2025-02-20
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-20


In [38]:
info_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         10 non-null     object        
 1   valoracion     10 non-null     float64       
 2   precio         10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 452.0+ bytes


In [39]:
info_competencia["id_hotel"] = ids_competencia
info_competencia["ciudad"] = "Madrid"
info_competencia

,nombre,valoracion,precio,fecha_reserva,id_hotel,ciudad
0,ibis Styles Madrid Prado,4.7,179.0,2025-02-20,113,Madrid
1,ibis budget Madrid Calle 30,4.4,106.0,2025-02-20,194,Madrid
2,ibis Madrid Centro las Ventas,4.5,181.0,2025-02-20,131,Madrid
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-20,114,Madrid
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-20,103,Madrid
5,ibis Madrid Aeropuerto Barajas,4.4,129.0,2025-02-20,181,Madrid
6,ibis Madrid Alcorcon Tresaguas,4.4,95.0,2025-02-20,128,Madrid
7,ibis budget Madrid Aeropuerto,4.0,97.0,2025-02-20,186,Madrid
8,ibis Madrid Alcobendas,4.4,89.0,2025-02-20,135,Madrid
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-20,117,Madrid


In [40]:
info_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         10 non-null     object        
 1   valoracion     10 non-null     float64       
 2   precio         10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
 4   id_hotel       10 non-null     int64         
 5   ciudad         10 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 612.0+ bytes


In [41]:
df_competencia = info_competencia[["id_hotel", "nombre", "valoracion", "ciudad"]]
competencia = True
df_competencia["competencia"] = competencia
df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)
df_competencia

C:\Users\marta\AppData\Local\Temp\ipykernel_6676\3554160467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia["competencia"] = competencia
C:\Users\marta\AppData\Local\Temp\ipykernel_6676\3554160467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)


,id_hotel,nombre_hotel,valoracion,ciudad,competencia
0,113,ibis Styles Madrid Prado,4.7,Madrid,True
1,194,ibis budget Madrid Calle 30,4.4,Madrid,True
2,131,ibis Madrid Centro las Ventas,4.5,Madrid,True
3,114,ibis budget Madrid Centro las Ventas,4.3,Madrid,True
4,103,ibis budget Madrid Vallecas,4.3,Madrid,True
5,181,ibis Madrid Aeropuerto Barajas,4.4,Madrid,True
6,128,ibis Madrid Alcorcon Tresaguas,4.4,Madrid,True
7,186,ibis budget Madrid Aeropuerto,4.0,Madrid,True
8,135,ibis Madrid Alcobendas,4.4,Madrid,True
9,117,ibis budget Madrid Alcorcon Móstoles,4.5,Madrid,True


In [42]:
df_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      10 non-null     int64  
 1   nombre_hotel  10 non-null     object 
 2   valoracion    10 non-null     float64
 3   ciudad        10 non-null     object 
 4   competencia   10 non-null     bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 462.0+ bytes


In [43]:
#unimos los dos dataframes de hoteles
df_hoteles = pd.concat([df_propios, df_competencia], axis = 0)
df_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 0 to 9
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre_hotel  29 non-null     object 
 1   id_hotel      29 non-null     int64  
 2   valoracion    29 non-null     float64
 3   ciudad        29 non-null     object 
 4   competencia   29 non-null     bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 1.2+ KB


In [44]:
valoracion_hoteles = generar_mapa("nombre_hotel", "valoracion", df_hoteles)
nombres_id_hoteles = generar_mapa("nombre_hotel", "id_hotel", df_hoteles)
nombres_competencia = generar_mapa("id_hotel", "nombre_hotel", df_competencia)
precios_competencia = generar_mapa("nombre", "precio", info_competencia)

## Clientes

Como podemos tener dos clientes con el mismo nombre, tomaremos el mail de cliente como el valor único de cliente. Es posible que tengamos un mismo cliente que haya hecho vrias reservas, por lo que encontrar el mismo id_cliente varias veces tampoco indicaría error. Vamos a comprobar si el número de valores únicos de id_cliente se corresponde con el número de valores únicos de mail. De no ser así, podría indicar un error en la asignación de id_cliente que debemos corregir

In [45]:
df["id_cliente"].nunique()

14847

In [46]:
df["mail"].nunique()

14905

Vemos que hay más valores de mail que de id_cliente, lo que sugiere que más de un cliente tiene el mismo id. Vamos a corregir el id de la misma forma que hemos corregido los id_hotel.

In [47]:
df_clientes = df[["id_cliente", "nombre", "apellido", "mail"]]
df_clientes.sample(5)

,id_cliente,nombre,apellido,mail
4777,ec8376f4-c69b-4c94-b58e-8db374a197c1,Joaquina,Diaz,joaquina.diaz@example.com
10242,4afc9500-6ae5-47d7-920d-c87a350ed697,Santos,Pagès,santos.pagès@example.com
3399,d3edc9a4-38bf-41d9-a2a2-a8d45b951e70,Marcos,Villalba,marcos.villalba@example.com
308,f04c9670-fa9d-4923-943d-58be45d2b854,María,Alcalá,maría.alcalá@example.com
9399,8ca4dbb4-8b0c-447b-8687-93c9c4d1caaf,Eloísa,Águila,eloísa.águila@example.com


In [48]:
mails=df_clientes["mail"].unique()

In [49]:
mailcliente=[]
idcliente=[]
for mail in enumerate(mails, start=1):
    mailcliente.append(mail[1])
    idcliente.append(f"cliente_{mail[0]}")
mail_cliente = dict(zip(mailcliente, idcliente))

In [50]:
df_clientes.loc[:, "id_cliente"] = df_clientes["mail"].map(mail_cliente)
df_clientes.sample(5)

,id_cliente,nombre,apellido,mail
4951,cliente_4941,Pastor,Badía,pastor.badía@example.com
355,cliente_356,Onofre,Martínez,onofre.martínez@example.com
4715,cliente_4707,Enrique,Soriano,enrique.soriano@example.com
5964,cliente_5949,Loreto,Criado,loreto.criado@example.com
13630,cliente_13554,Antonio,Coronado,antonio.coronado@example.com


In [51]:
df_clientes = df_clientes.drop_duplicates()
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14905 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  14905 non-null  object
 1   nombre      14905 non-null  object
 2   apellido    14905 non-null  object
 3   mail        14905 non-null  object
dtypes: object(4)
memory usage: 582.2+ KB


In [52]:
df_clientes.duplicated().sum()

np.int64(0)

## Reservas

In [53]:
# Renombramos la columna estrellas por valoracion
df.rename(columns = {"estrellas": "valoracion"}, inplace = True)

In [54]:
# La fecha de reserva para los hoteles de la competencia es la del scrapeo. Lo actualizamos
df.loc[df["competencia"]==True, "fecha_reserva"] = info_competencia["fecha_reserva"][0]
df["id_cliente"] = df["mail"].map(mail_cliente)
df.loc[df["competencia"]==False, "id_hotel"] = df["nombre_hotel"].map(hoteles_propios)
df.loc[df["competencia"] == True, "nombre_hotel"] = df["id_hotel"].map(nombres_competencia)
df.loc[df["competencia"] == True, "precio_noche"] = df["nombre_hotel"].map(precios_competencia)
df["valoracion"] = df["nombre_hotel"].map(valoracion_hoteles)

In [55]:
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,valoracion,ciudad
11828,a04b03a5-db22-4a85-8588-9fadd1fada76,cliente_11776,Paloma,Baena,paloma.baena@example.com,False,2025-02-01,2025-03-01,2025-03-02,3,495.02,Hotel Camino del Sol,3.1,Madrid
10408,0d5d74ef-39f1-4f9e-b18d-ec8f0f6c71a3,cliente_10364,Inocencio,Sales,inocencio.sales@example.com,False,2025-02-11,2025-03-01,2025-03-02,8,198.61,Hotel Las Estrellas,2.9,Madrid
2877,c63965b7-0e52-44bf-8720-d089a024f0d3,cliente_2877,Santos,Salmerón,santos.salmerón@example.com,True,2025-02-20,2025-03-01,2025-03-02,113,179.00,ibis Styles Madrid Prado,4.7,Madrid
516,5ddb6dae-5004-4740-9a15-c59e8cdf9c08,cliente_517,Dafne,Cid,dafne.cid@example.com,True,2025-02-20,2025-03-01,2025-03-02,181,129.00,ibis Madrid Aeropuerto Barajas,4.4,Madrid
7621,bb5a4749-fa16-4e5d-894f-a42fd5e1c4ce,cliente_7593,Jose Manuel,Pozo,jose manuel.pozo@example.com,True,2025-02-20,2025-03-01,2025-03-02,128,95.00,ibis Madrid Alcorcon Tresaguas,4.4,Madrid


In [56]:
df_reservas = df[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel", "nombre_hotel"]]
df_reservas.sample(10)

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel,nombre_hotel
13374,1f60c87c-48b2-49a9-924e-51000420ebfd,2025-02-01,2025-03-01,2025-03-02,120.58,cliente_13305,16,Hotel Rincón Sereno
8498,5dbf9656-468e-423b-82ae-faeda522a2f2,2025-02-07,2025-03-01,2025-03-02,450.40,cliente_8466,10,Hotel Torre Dorada
7971,eec23195-6cee-459e-a879-37b13de6497b,2025-02-03,2025-03-01,2025-03-02,65.19,cliente_7941,11,Hotel Palacio Imperial
2954,699cde9f-1a7b-47e3-b514-142d1de8336d,2025-02-05,2025-03-01,2025-03-02,164.08,cliente_2954,11,Hotel Palacio Imperial
809,b92df29b-0569-41dd-865c-c70ad48c83b1,2025-02-10,2025-03-01,2025-03-02,464.41,cliente_810,17,Hotel Vista Alegre
10167,e4f30dc1-4b17-4a98-9d37-0c8a65b1d0bf,2025-02-11,2025-03-01,2025-03-02,55.66,cliente_10127,18,Hotel Costa Azul
8181,20082155-6123-426e-aebe-3d40fd700869,2025-02-03,2025-03-01,2025-03-02,180.96,cliente_8150,1,Hotel Monte Verde
11053,acdd9964-5726-4bf3-a032-6a0f3fa4b788,2025-02-09,2025-03-01,2025-03-02,440.16,cliente_11005,9,Gran Hotel Madrid
3888,ba34ca93-54c6-4454-a358-f149d847c8c8,2025-02-04,2025-03-01,2025-03-02,52.83,cliente_3884,8,Hotel Las Estrellas
13555,143cb526-e018-4812-a386-330c6917901d,2025-02-20,2025-03-01,2025-03-02,102.00,cliente_13480,103,ibis budget Madrid Vallecas


In [57]:
df_reservas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   fecha_reserva    15000 non-null  datetime64[ns]
 2   inicio_estancia  15000 non-null  datetime64[ns]
 3   final_estancia   15000 non-null  datetime64[ns]
 4   precio_noche     14948 non-null  float64       
 5   id_cliente       15000 non-null  object        
 6   id_hotel         15000 non-null  int64         
 7   nombre_hotel     15000 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(3)
memory usage: 1.5+ MB


In [58]:
df_precios = df_reservas.groupby("id_hotel")["precio_noche"].describe()
df_precios["IQR"] =df_precios["75%"] - df_precios["25%"]
df_precios["limite_inferior"] = df_precios["25%"] - 1.5 * df_precios["IQR"]
df_precios["limite_superior"] = df_precios["75%"] + 1.5 * df_precios["IQR"]
df_precios

,count,mean,std,min,25%,50%,75%,max,IQR,limite_inferior,limite_superior
id_hotel,,,,,,,,,,,
1,546.0,276.612381,130.503215,50.01,166.9450,275.020,395.8350,497.76,228.8900,-176.39000,739.17000
2,532.0,275.225921,128.611561,50.43,158.2000,283.895,379.6600,499.73,221.4600,-173.99000,711.85000
3,514.0,269.998444,129.969971,51.09,158.7400,271.295,385.0225,499.27,226.2825,-180.68375,724.44625
4,523.0,280.151243,128.943986,53.29,172.5900,282.050,394.9800,497.83,222.3900,-160.99500,728.56500
5,508.0,278.243996,131.498543,51.83,163.6500,274.610,394.8650,497.65,231.2150,-183.17250,741.68750
6,507.0,282.492150,130.337947,52.18,175.6300,279.540,399.7750,499.41,224.1450,-160.58750,735.99250
7,501.0,272.402335,133.457176,51.99,161.2900,265.120,389.9500,499.63,228.6600,-181.70000,732.94000
8,510.0,271.706353,128.303521,50.19,166.0875,264.745,380.4075,499.82,214.3200,-155.39250,701.88750
9,515.0,264.967165,127.484497,50.27,155.8550,260.080,371.3550,498.59,215.5000,-167.39500,694.60500


In [59]:
ids_hoteles = range(1,20)
for id in ids_hoteles:
    valor = df_reservas[df_reservas["id_hotel"] == id]["precio_noche"] 
    outliers = (valor > df_precios.loc[id, "limite_superior"]) | (valor < df_precios.loc[id, "limite_inferior"])
    num_outliers = outliers.sum()
    total_values = len(valor)
    percentage_outliers = (num_outliers / total_values) * 100

    # Mostrar resultados
    print({id})
    print(f'Cantidad de outliers: {num_outliers}')
    print(f'Porcentaje de outliers: {percentage_outliers:.2f}%')

{1}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{2}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{3}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{4}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{5}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{6}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{7}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{8}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{9}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{10}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{11}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%


{12}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{13}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{14}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{15}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{16}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{17}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{18}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%
{19}
Cantidad de outliers: 0
Porcentaje de outliers: 0.00%


In [60]:
# Creamos un diccionario con los precios medios de los hoteles para poder rellenar los valores nulos
precios_medios = dict(df_precios["mean"])
df.loc[df["competencia"]==False, "id_hotel"] = df["nombre_hotel"].map(hoteles_propios)
df_reservas.loc[df_reservas["precio_noche"].isnull(), "precio_noche"] = df_reservas["id_hotel"].map(precios_medios)
df_reservas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   fecha_reserva    15000 non-null  datetime64[ns]
 2   inicio_estancia  15000 non-null  datetime64[ns]
 3   final_estancia   15000 non-null  datetime64[ns]
 4   precio_noche     15000 non-null  float64       
 5   id_cliente       15000 non-null  object        
 6   id_hotel         15000 non-null  int64         
 7   nombre_hotel     15000 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(3)
memory usage: 1.5+ MB


In [66]:
#Guardamos los df generados en archivos pickle
df_ciudades.to_pickle("../data/data_clean/ciudades.pkl")
df_hoteles.to_pickle("../data/data_clean/hoteles.pkl")
df_clientes.to_pickle("../data/data_clean/clientes.pkl")
df_reservas.to_pickle("../data/data_clean/reservas.pkl")